In [135]:
import numpy as np
import pandas as pd

## Read the data 

In [159]:
train = pd.read_csv('data/task2_lemmas_train', error_bad_lines=False)
test = pd.read_csv('data/task2_lemmas_test', error_bad_lines=False)

Skipping line 102: expected 3 fields, saw 4
Skipping line 229: expected 3 fields, saw 4
Skipping line 297: expected 3 fields, saw 4
Skipping line 373: expected 3 fields, saw 4
Skipping line 399: expected 3 fields, saw 4
Skipping line 410: expected 3 fields, saw 4
Skipping line 414: expected 3 fields, saw 4
Skipping line 488: expected 3 fields, saw 4
Skipping line 501: expected 3 fields, saw 4
Skipping line 508: expected 3 fields, saw 4
Skipping line 563: expected 3 fields, saw 4
Skipping line 622: expected 3 fields, saw 4
Skipping line 712: expected 3 fields, saw 4
Skipping line 751: expected 3 fields, saw 4
Skipping line 787: expected 3 fields, saw 4
Skipping line 804: expected 3 fields, saw 4
Skipping line 825: expected 3 fields, saw 4
Skipping line 886: expected 3 fields, saw 4
Skipping line 919: expected 3 fields, saw 4
Skipping line 1043: expected 3 fields, saw 4
Skipping line 1109: expected 3 fields, saw 4
Skipping line 1141: expected 3 fields, saw 4
Skipping line 1190: expected 

In [160]:
train[:3]

,Id,X,y
0,1,vergognerete,vergognare+V
1,2,amnistiavate,amnistiare+V
2,3,menomazione,menomazione+N


## Feature engineering 

In [161]:
train.drop('Id', axis=1, inplace=True)

In [162]:
train['Y'] = train['y'].apply(lambda x: x[:-2])

In [163]:
train['PoS_y'] = train['y'].apply(lambda x: x[-1:])

In [164]:
train.rename(columns={'y': 'answers'}, inplace=True)

In [225]:
%%time
verbs = {}
nouns = {}
adjs = {}

for i, row in train.iterrows():
    min_len_word = len(row['Y']) if len(row['Y']) < len(row['X']) else len(row['X'])
    for k in range(min_len_word):
        if(row['X'][k] != row['Y'][k]):
            end_x = row['X'][k:]
            end_y = row['Y'][k:]
            break
    
    if (row['PoS_y'] == 2):
        verbs[end_x] = end_y
    elif (row['PoS_y'] == 1):
        nouns[end_x] = end_y
    else:
        adjs[end_x] = end_y

CPU times: user 41.2 s, sys: 396 ms, total: 41.5 s
Wall time: 41.1 s


In [229]:
verbs

464

In [165]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
train['PoS_y'] = encoder.fit_transform(train['PoS_y'])

In [183]:
train[:3]

,X,answers,Y,PoS_y
0,vergognerete,vergognare+V,vergognare,2
1,amnistiavate,amnistiare+V,amnistiare,2
2,menomazione,menomazione+N,menomazione,1


In [147]:
%%time
from sklearn.feature_extraction.text import CountVectorizer

ngram_vectorizer = CountVectorizer(
    analyzer='char_wb', 
    ngram_range=(1, 7), 
    max_features=200000,
    min_df=1, 
    lowercase=False
) 

ngram_train = ngram_vectorizer.fit_transform(train['X'])
# best = 200k features

CPU times: user 12.1 s, sys: 124 ms, total: 12.3 s
Wall time: 12.3 s


## Cross Validation 

In [148]:
from sklearn.cross_validation import train_test_split

xtr, xcv, ytr, ycv = train_test_split(ngram_train, train['PoS_y'], test_size = 0.1,  random_state = 42)

#### Logistic Regression 

In [149]:
%%time
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(
    penalty='l2',
    dual=True,
    solver='liblinear'
)

lr.fit(xtr, ytr)
lr_preds = lr.predict(xcv)

CPU times: user 33.3 s, sys: 0 ns, total: 33.3 s
Wall time: 33.4 s


In [150]:
from sklearn.metrics import accuracy_score
accuracy_score(ycv, lr_preds)

0.97891594365146006

The best is: 0.97965315934065933